### Check Version of the Relevant Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Overfitting the test set

- Import `breast_cancer` dataset from Scikitlearn
- Scale the feature set
- Split data into first trainval and test, then split trainval into validation and train datasets
- Fit train dataset for `k=5` neighbors
- print the score on validation and test datasets

In [13]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

data = load_breast_cancer()
X = data.data
y = data.target

# Scale the feature set
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split into trainval and test
X_trainval, X_test, y_trainval, y_test = train_test_split(X_scaled, y, random_state=0)

#split trainval into train and validation
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, random_state=0)

X_trainval.shape, X_train.shape, X_val.shape, X_test.shape, X.shape

# fit the train dataset for k=5 neighbors
knn = KNeighborsClassifier(n_neighbors = 5)
knn.fit(X_train, y_train)
print('Validation score is:', knn.score(X_val, y_val))
print('Test score is: ', knn.score(X_test, y_test))

Validation score is: 0.9719626168224299
Test score is:  0.9440559440559441


- Create a loop and fit your train dataset and test your validation dataset for `k=1,15,2`.
- Print the best validation score out of three neighbors and determine the `k` value with the highest score, $best_n$
- Retrain your model with trainvalidation dataset for $k=best_n$
- Print the score for test dataset for the latest trained model

In [21]:
nset = [1,15,2]
for n in nset:
  knn = KNeighborsClassifier(n_neighbors=n)
  knn.fit(X_train, y_train)
  print(f'Validation score for {n} neighbors is: ', knn.score(X_val, y_val))
  print(f'Test score for {n} neighbors is: ', knn.score(X_test, y_test))

# Retrain your model with trainvalidation dataset k = bestn
knn = KNeighborsClassifier(n_neighbors=15)
knn.fit(X_trainval, y_trainval)

print('Score for optimised model = ', knn.score(X_test, y_test))

Validation score for 1 neighbors is:  0.9439252336448598
Test score for 1 neighbors is:  0.9370629370629371
Validation score for 15 neighbors is:  0.9532710280373832
Test score for 15 neighbors is:  0.958041958041958
Validation score for 2 neighbors is:  0.9439252336448598
Test score for 2 neighbors is:  0.916083916083916
Score for optimised model =  0.951048951048951


# Cross validation

- Import `cross_val_score` from `sklearn.model_selection`
- Split your data into train and test datasets
- For `neighbors=1, 15, 2`, compute cross validation scores for trainvalidation dataset with kfold=10.
- Print the best validation score out of three neighbors and determine the `k` value with the highest score, $best_n$
- Retrain your model with trainvalidation dataset for $k=best_n$
- Print the score for test dataset for the latest trained model

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)


- Compute cross validation score for `KNeighborsClassifier()` when `cv=KFold(n_splits=5)` and `cv=StratifiedKFold(n_splits=5, shuffle=True)`

In [ ]:
np.set_printoptions(precision=2)

# GridSearchCV

- Import `GridSearchCV` from `sklearn.model_selection`
- Split your data into train and test datasets
- For `neighbors=1 to 30`, compute `GridSearchCV` for train dataset with kfold=10.
- Print the best cross validation score
- Priont the best parameter
- Print the test score